In [1]:
import pandas as pd

df=pd.read_excel(r'/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/Updated_full_Filtered_Data_with_real_length.xlsx')

print(len(df))
df=df.dropna()
print(len(df))

# df['Pond_Type'] = df['Pond_Type'].replace({'test-car': 'square-femal', 'test-right': 'circular big male', 'test-left': 'circular small male'})
df['Pond_Type'].unique()

132
118


array(['right', 'left', 'car'], dtype=object)

In [2]:
# First, let's see what unique values we have before replacement
print("Before replacement:", df['Pond_Type'].unique())

# Then do a complete replacement that covers all cases
df['Pond_Type'] = df['Pond_Type'].replace({
    'car': 'square',
    # 'test-right': 'circle2',
    'right': 'circle2',  # Added this to handle 'right'
    'left': 'circle1',
    # 'test-left': 'circle1'
})

print("After replacement:", df['Pond_Type'].unique())

Before replacement: ['right' 'left' 'car']
After replacement: ['circle2' 'circle1' 'square']


In [3]:
# left:circl2 ; right: circl2 ; car: square




df['Pond_Type'] = df['Pond_Type'].replace({'car': 'square', 'test-right': 'circl2', 'left': 'circl1'})
df['Pond_Type'].unique()

array(['circle2', 'circle1', 'square'], dtype=object)

In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, shapiro, levene
# import pingouin as pg
import statsmodels.api as sm
import numpy as np
import plotly.express as px

# Load your data

# Calculate expert averages and standard deviations
df['expert_mean'] = df[['Length_1', 'Length_2', 'Length_3']].median(axis=1)
df['expert_std'] = df[['Length_1', 'Length_2', 'Length_3']].std(axis=1)

df['expert_max'] = df[['Length_1', 'Length_2', 'Length_3']].max(axis=1)
df['expert_min'] = df[['Length_1', 'Length_2', 'Length_3']].min(axis=1)

#diffrence in pixels between the min and max of Length_1_pixels, Length_2_pixels, Length_3_pixels

df['diff_max_min'] =abs(df[['Length_1', 'Length_2', 'Length_3']].max(axis=1) - df[['Length_1', 'Length_2', 'Length_3']].min(axis=1))

print(f'mean difference between min max' ,df['diff_max_min'].mean())
print(f'std  difference between min max ',df['diff_max_min'].std())
#
median_diff_max_min = df['diff_max_min'].median()
print(f'median difference between min max', median_diff_max_min)

#violoin plot of the difference between the min and max of the three measurements
fig = px.violin(df,
                x='Pond_Type', 
                y='diff_max_min', 
                box=True,  # Include box plot
                points="all",  # Show all points
                hover_data=['Label', 'PrawnID'],  # Include hover data
                title=f'Violin plot of diff_max_min in mm')
#Save the plot to png
# fig.write_image(f'violin carapace diff_max_min.png')
fig.show()

#how many Length_ground_truth_annotation(mm) in comparison with lengthe_1, length_2, length_3 bigger than 50%

# df['difference_mean_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_mean']/df['Length_ground_truth_annotation(mm)']) 
# df['difference_max_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_max']/df['Length_ground_truth_annotation(mm)'])
# df['difference_min_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_min']/df['Length_ground_truth_annotation(mm)'])

# df['min difference'] = df[['difference_mean_annotation_expert', 'difference_max_annotation_expert', 'difference_min_annotation_expert']].min(axis=1)
# df['max_difference'] = df[['difference_mean_annotation_expert', 'difference_max_annotation_expert', 'difference_min_annotation_expert']].max(axis=1)    


# df['difference_max_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_max']/df['Length_ground_truth_annotation(mm)'])

# df['difference_min_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_min']/df['Length_ground_truth_annotation(mm)'])
#violin difference_annotation_expert
# fig = px.violin(df,
#                 x='Pond_Type',
#                 y='min difference',
#                 box=True,  # Include box plot
#                 points="all",  # Show all points
#                 hover_data=['Label', 'PrawnID'],  # Include hover data
#                 title=f'Violin plot of difference_annotation_expert',)
# #Save the plot to png
# fig.write_image(f'violin carapace difference_annotation_expert.png')
# fig.show()




# Calculate differences
# df['diff_mean'] =abs(df['Length_fov(mm)'] - df['expert_mean'])
# df['diff_max'] =abs(df['Length_fov(mm)'] - df['expert_max'])
# df['diff_min'] =abs(df['Length_fov(mm)'] - df['expert_min'])

# df['the_min_diff'] = df[['diff_mean', 'diff_max', 'diff_min']].min(axis=1)


#the length measurement of the the_min_diff
# df['the_min_diff_length'] =df['Length_fov(mm)'] - df['the_min_diff']

#the length measurement of the the_max_diff
#percent error of the the_min_diff


# df['org_min_diff'] = [
#     df['Length_fov(mm)'].iloc[i] - df['the_min_diff'].iloc[i] 
#     if df['the_min_diff'].iloc[i] > 0 
#     else df['Length_fov(mm)'].iloc[i] + abs(df['the_min_diff'].iloc[i]) 
#     for i in range(len(df))
# ]


# df['min difference']=df[['Error_percentage_distance_mm_ground_min', 'Error_percentage_distance_mm_ground_max','Error_percentage_distance_mm_ground_median']].min(axis=1)

# Calculate the percentage error
# df['the_min_diff_percentage_error'] = (df['the_min_diff'].abs() / df['org_min_diff'].abs()) * 100

# # Cap the error at 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error'].apply(lambda x: min(x, 100))

# the percentage error of the the_min_diff when smaller than 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error']
# df['min difference']=df[['Error_percentage_distance_mm_ground_min', 'Error_percentage_distance_mm_ground_max','Error_percentage_distance_mm_ground_median']].min(axis=1)

# Calculate the percentage error
# df['the_min_diff_percentage_error'] = (df['the_min_diff'].abs() / df['org_min_diff'].abs()) * 100

# # Cap the error at 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error'].apply(lambda x: min(x, 100))

# the percentage error of the the_min_diff when smaller than 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error']
df['min_mpe']=df[['MPError_fov_min', 'MPError_fov_max','MPError_fov_median']].min(axis=1)

df['max_mpe']=df[['MPError_fov_min', 'MPError_fov_max','MPError_fov_median']].max(axis=1)

df['min_mae']=df[['AbsError_fov_min', 'AbsError_fov_max','AbsError_fov_median']].min(axis=1)

df['max_mae']=df[['AbsError_fov_min', 'AbsError_fov_max','AbsError_fov_median']].max(axis=1)




print('max mae median:')
print(df.groupby('Pond_Type')['max_mae'].median())

#max mpe median
print('max mpe median:')
print(df.groupby('Pond_Type')['max_mpe'].median())


# Create a violin plot using plotly
import plotly.express as px
#violin plots wiht plotly for mean percentage error and MAE
for error in ['min_mpe', 'min_mae', 'max_mpe', 'max_mae']:
        # Create a violin plot using plotly 
    fig = px.violin(df,
                    x='Pond_Type', 
                    y=error, 
                    box=True,  # Include box plot
                    points="all",  # Show all points
                    hover_data=['Label', 'PrawnID'],  # Include hover data
                    title=f'Violin plot of {error} in mm' )
    #Save the plot to png
    # fig.write_image(f'violin full {error}_.png')
    fig.show()

#change name of each pond type if test-car change to square-female if test-right change to circular big male if test-left change to circular small mal

print('min mae median:')
print("min mae for all pond types:")
print(df['min_mae'].median())
print("min mae for each pond type:")
print(df.groupby('Pond_Type')['min_mae'].median())

#min mpe median
print('min mpe median:')
print("min mpe for all pond types:")
print(df['min_mpe'].median())
print("min mpe for each pond type:")
print(df.groupby('Pond_Type')['min_mpe'].median())


# #scatter plot the_min_diff_percentage_error and  difference_annotation_expert plotly
# fig = px.scatter(df, x='min_mpe', y='min difference', color='Pond_Type', hover_data=['Label', 'PrawnID'])
# #add title
# fig.update_layout(title='min_mpe vs. difference_annotation_expert')

# #save the plot to png
# fig.write_image('the_min_diff_percentage_error vs. difference_annotation_expert.png')
# # Add a trendline


# fig.show()


# #scatter plot the_min_diff_percentage_error and  pose_eval_iou plotly
# fig = px.scatter(df, x='min_mpe', y='pose_eval_iou', color='Pond_Type', hover_data=['Label', 'PrawnID'])
# #add title
# fig.update_layout(title='min_mpe vs. pose_eval_iou')

# #save the plot to png
# fig.write_image('the_min_diff_percentage_error vs. pose_eval_iou.png')
# # Add a trendline
# fig.show()

#scatter plot between min_mpe and height
fig = px.scatter(df, x='Height_mm', y='min_mpe', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mpe vs. Height')

#save the plot to png
fig.write_image('full body_ min_mpe vs. Height.png')
# Add a trendline
fig.show()

#max mae median

#total min mae



# Normality Test


mean difference between min max 13.794686440677964
std  difference between min max  8.681768663300264
median difference between min max 12.354000000000006


max mae median:
Pond_Type
circle1    18.316666
circle2    18.473935
square     21.475770
Name: max_mae, dtype: float64
max mpe median:
Pond_Type
circle1    28.875155
circle2    16.338081
square     17.982039
Name: max_mpe, dtype: float64


min mae median:
min mae for all pond types:
6.089788089166753
min mae for each pond type:
Pond_Type
circle1    8.683780
circle2    5.651378
square     5.188104
Name: min_mae, dtype: float64
min mpe median:
min mpe for all pond types:
5.151958196860564
min mpe for each pond type:
Pond_Type
circle1    12.110762
circle2     4.767052
square      5.154757
Name: min_mpe, dtype: float64


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Label', 'PrawnID', 'Avg_Length', 'Std_Length', 'Uncertainty', 'Length_1', 'Length_2', 'Length_3', 'BoundingBox_1', 'BoundingBox_2', 'BoundingBox_3', 'Angle_1', 'Angle_2', 'Angle_3', 'Scale_1', 'Scale_2', 'Scale_3', 'Global_Uncertainty', 'Length_ground_truth_annotation(mm)', 'id', 'Length_fov(mm)', 'Height(mm)', 'focal_RealLength(cm)', 'Length_1_pixels', 'Length_2_pixels', 'Length_3_pixels', 'Pond_Type', 'Euclidean_Distance', 'MPError_fov_min', 'MPError_fov_max', 'MPError_fov_median', 'AbsError_fov_min', 'AbsError_fov_max', 'AbsError_fov_median', 'AbsError_focal_min', 'AbsError_focal_max', 'AbsError_focal_median', 'MPError_focal_min', 'MPError_focal_max', 'MPError_focal_median', 'Error_distance_mm_ground_min', 'Error_distance_mm_ground_max', 'Error_distance_mm_ground_median', 'Error_percentage_distance_mm_ground_min', 'Error_percentage_distance_mm_ground_max', 'Error_percentage_distance_mm_ground_med

In [11]:
#avg length
print('avg length:')
print(df.groupby('Pond_Type')['Avg_Length'].median())


avg length:
Pond_Type
car      112.664833
left      68.095333
right    116.280667
Name: Avg_Length, dtype: float64


In [14]:
#length range between min and max
print('length range between min and max:')
print(df.groupby('Pond_Type').agg({
    'Length_1': ['min', 'max'],
    'Length_2': ['min', 'max'], 
    'Length_3': ['min', 'max']
}).round(2))


length range between min and max:
          Length_1         Length_2         Length_3        
               min     max      min     max      min     max
Pond_Type                                                   
car          89.03  144.13    86.33  137.24    95.56  140.43
left         36.90  135.61    34.40  134.93    39.16  126.32
right        68.78  169.07    63.03  171.18    70.58  166.31


In [18]:
#median and std of the length

import scipy.stats as stats

from scipy.stats import median_abs_deviation

print('median and median absolute deviation of the length:')
print(df.groupby('Pond_Type').agg({
    'Length_1': [lambda x: np.median(x), lambda x: median_abs_deviation(x, scale='normal')],
    'Length_2': [lambda x: np.median(x), lambda x: median_abs_deviation(x, scale='normal')],
    'Length_3': [lambda x: np.median(x), lambda x: median_abs_deviation(x, scale='normal')]
}).round(2))



median and median absolute deviation of the length:
            Length_1              Length_2              Length_3           
          <lambda_0> <lambda_1> <lambda_0> <lambda_1> <lambda_0> <lambda_1>
Pond_Type                                                                  
car           109.16      11.85     111.73      18.69     113.49      20.64
left           64.54      22.74      68.79      19.66      71.72      19.98
right         113.84      18.84     112.24      20.75     120.17      20.82


In [19]:
df['Mean_Length'] = df[['Length_1', 'Length_2', 'Length_3']].mean(axis=1)
df['Std_Dev_Length'] = df[['Length_1', 'Length_2', 'Length_3']].std(axis=1)

# Derive overall uncertainty by pond type
overall_stats = df.groupby('Pond_Type').agg({
    'Mean_Length': 'mean',
    'Std_Dev_Length': 'mean'
}).rename(columns={
    'Mean_Length': 'Overall_Mean_Length',
    'Std_Dev_Length': 'Overall_Uncertainty'
}).reset_index()

# Print per-prawn uncertainty
print("Per-Prawn Uncertainty:")
# print(df[['Pond_Type', 'Mean_Length', 'Std_Dev_Length']].round(2))

# Print overall uncertainty by pond type
print("\nOverall Uncertainty by Pond Type:")
print(overall_stats.round(2))

Per-Prawn Uncertainty:

Overall Uncertainty by Pond Type:
  Pond_Type  Overall_Mean_Length  Overall_Uncertainty
0       car               114.17                 7.11
1      left                73.02                 5.27
2     right               116.34                 7.53


In [48]:
fig = px.scatter(df,color='PondType', x='min_mpe', y='min_mpe_pixels',hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title=f'min_mpe vs. min_mpe_pixels')

#save the plot to png
fig.write_image(f'full body_min_mpe vs. min_mpe_pixels.png')
# Add a trendline
fig.show()

In [47]:
#relationship between min_mpe and min_mpe_of_pixels
for pond_type in df['PondType'].unique():

    fig = px.scatter(df[df['PondType'] == pond_type], x='min_mpe', y='min_mpe_pixels',hover_data=['Label', 'PrawnID'])
    #add title
    fig.update_layout(title=f'min_mpe vs. min_mpe_pixels {pond_type}')

    #save the plot to png
    fig.write_image(f'full body_min_mpe vs. min_mpe_pixels {pond_type}.png')
    # Add a trendline
    fig.show()


In [11]:
fig = px.scatter(df, x='Height_mm', y='min_mae', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mae vs. Height')

#save the plot to png
fig.write_image('full body_ min_mae vs. Height.png')
# Add a trendline
fig.show()

In [15]:
# a graph showing overestimation and underestimation of the length of the prawn
df['overestimation'] = df['Length_FOV_box'] - df['expert_max']
#show the overestimation of the length of the prawn
fig = px.scatter(df, x='Height_mm', y='overestimation', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='overestimation vs. Height')

#save the plot to png
fig.write_image('full body_ overestimation vs. Height.png')
# Add a trendline
fig.show()



In [23]:
#groupby pond type unique label
df.groupby('PondType')['Label'].nunique()

PondType
circular-big      33
circular-small     5
square-female     13
Name: Label, dtype: int64

In [11]:
import plotly.express as px





df['min_mpe_length-'] = df.apply(lambda row: row['Length_FOV_box'] - row['Length_FOV_box'] * row['min_mpe']/100, axis=1)
                                
df['min_mpe_length+']= df.apply(lambda row: row['Length_FOV_box'] + row['Length_FOV_box'] * row['min_mpe']/100, axis=1)

df['min_mae_length-'] = df.apply(lambda row: row['Length_FOV_box'] -  row['min_mae'], axis=1)
df['min_mae_length+'] = df.apply(lambda row: row['Length_FOV_box'] +  row['min_mae'], axis=1)




def get_closest_length_and_distance(row, target_length1,true_length2):
    lengths = [row['Length_1'], row['Length_2'], row['Length_3']]
    closest_length_1 = min(lengths, key=lambda x: abs(x - target_length1))
    distance_1 = abs(closest_length_1 - target_length1)
    closest_length_2 = min(lengths, key=lambda x: abs(x - true_length2))
    distance_2 = abs(closest_length_2 - true_length2)

    if distance_1 < distance_2:
        closest_length = closest_length_1
        distance = distance_1
    else:
        closest_length = closest_length_2
        distance = distance_2


    return closest_length, distance

# Apply the function to find the closest length and distance to min_mpe_length-
df['closest_length_min_mpe'] = df.apply(
    lambda row: (get_closest_length_and_distance(row, row['min_mpe_length-'],row['min_mpe_length+'])), axis=1
)

df['closest_length_min_mae'] = df.apply( lambda row: (get_closest_length_and_distance(row, row['min_mae_length-'],row['min_mae_length+'])), axis=1)

# Apply the function to find the closest length and distance to min_mpe_length+
df['closest_length_min_mae_value']=df['closest_length_min_mae'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
# Assuming df contains the columns 'predicted_length' and 'true_length'
#take the overestimation from the min_percentage_error length
# find the true length when min_mpe is used

#min_mpe_length will be decided by the min_mpe percentage error 


df['closest_length_min_mpe_value'] = df['closest_length_min_mpe'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

# Calculate overestimation
df['overestimation'] = df['Length_FOV_box'] - df['closest_length_min_mae_value']














                                 #approximatly equal to length_1 

# df['overestimation'] = df['Length_FOV_box'] - df['closest_length_min_mpe']






for pond_type in df['PondType'].unique():
    fig = px.histogram(df[df['PondType'] == pond_type], x='overestimation', nbins=30, marginal="box", title=f"Histogram of Residuals {pond_type} Overestimation ",hover_data=['Label', 'PrawnID'])
    fig.update_layout(
    xaxis_title=f"Residuals {pond_type} (Overestimation vs Underestimation)",
    yaxis_title="Frequency",
    )
    # Add a vertical line at 0 to show the separation between overestimation and underestimation
    fig.add_vline(x=0, line_dash="dash", line_color="red")

    # Save the plot to a file
    fig.write_image(f'histogram_residuals {pond_type}.png')

    fig.show()

# df['underestimation'] = df['Length_FOV_box'] - df['expert_min']

# for pond_type in df['PondType'].unique():
#     fig = px.histogram(df[df['PondType'] == pond_type], x='underestimation', nbins=30, marginal="box", title=f"Histogram of Residuals {pond_type}  Underestimation)",hover_data=['Label', 'PrawnID'])
#     fig.update_layout(
#     xaxis_title=f"Residuals {pond_type} (Overestimation vs Underestimation)",
#     yaxis_title="Frequency",
#     )
#     # Add a vertical line at 0 to show the separation between overestimation and underestimation
#     fig.add_vline(x=0, line_dash="dash", line_color="red")

#     # Save the plot to a file
#     fig.write_image(f'histogram_residuals {pond_type}.png')

#     fig.show()

# Calculate overestimation and underestimation counts




In [12]:
import plotly.graph_objects as go


#scatter plot between length_fov and closest_length_min_mae_value
fig = px.scatter(df,color='PondType', x='Length_box_focal', y='closest_length_min_mae_value', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title=f'pinhole vs. closest_length_min_mae_value')
fig.add_trace(go.Scatter(x=[0, 200], y=[0, 200], mode='lines', name='line'))

#save the plot to png
fig.write_image(f'full focal vs. closest_length_min_mae_value.png')
#add a trendline

#add line


fig.show()
#

In [18]:
overestimations = len(df[df['overestimation'] > 0])
underestimations = len(df[df['overestimation'] < 0])

# Create a bar plot
fig = px.bar(
    x=['Overestimations', 'Underestimations'], 
    y=[overestimations, underestimations],
    title="Overestimations vs Underestimations"
)
fig.update_layout(
    yaxis_title="Count"
)
# Save the plot to a file
fig.write_image('overestimations_vs_underestimations.png')

fig.show()

In [40]:
    
    
    
for pond_type in df['PondType'].unique():    


    fig = px.ecdf(df[df['PondType'] == pond_type], x="overestimation", title=f"Cumulative Distribution of Residuals (CDF) {pond_type}",hover_data=['Label', 'PrawnID'])
    fig.update_layout(
        xaxis_title="Residuals (Predicted - True)"
    )
    # Add a vertical line at 0 to show the separation between overestimation and underestimation
    fig.add_vline(x=0, line_dash="dash", line_color="red")

    fig.show()


In [49]:
#min_mpe_focal_length from MPError_focal_min, MPError_focal_max, MPError_focal_median
# min_mae_focal_length from AbsError_focal_min, AbsError_focal_max, AbsError_focal_median

df['min_mpe_focal_length']=df[['Error_percentage_max_box_focal','Error_percentage_min_box_focal','Error_percentage_median_box_focal']].min(axis=1)

df['min_mae_focal_length']=df[['Abs_error_max_box_focal','Abs_error_min_box_focal','Abs_error_median_box_focal']].min(axis=1)

df['max_mpe_focal_length']=df[['Error_percentage_max_box_focal','Error_percentage_min_box_focal','Error_percentage_median_box_focal']].max(axis=1)

df['max_mae_focal_length']=df[['Abs_error_max_box_focal','Abs_error_min_box_focal','Abs_error_median_box_focal']].max(axis=1)



#print title of the prints
print('pond type count:')
print(df['PondType'].value_counts())

print('pond type mean:')
print(df.groupby('PondType')['Avg_Length'].mean())

print('pond type std:')
print(df.groupby('PondType')['Avg_Length'].std())


print ('min_mpe fov median:')
print(df.groupby('PondType')['min_mpe'].median())
#mean and std for each pond type of the lengt_1, length_2, length_

#print min_mae median and std for each pond type

print('min_mae fov median:')
print(df.groupby('PondType')['min_mae'].median())



print('max_mpe fov median:')
print(df.groupby('PondType')['max_mpe'].median())

print('max_mae fov median:')
print(df.groupby('PondType')['max_mae'].median())


print('min_mpe focal length median:')
print(df.groupby('PondType')['min_mpe_focal_length'].median())

print('max_mpe focal length median:')
print(df.groupby('PondType')['max_mpe_focal_length'].median()) 

print('min_mae focal length median:')
print(df.groupby('PondType')['min_mae_focal_length'].median())

print('max_mae focal length median:')   
print(df.groupby('PondType')['max_mae_focal_length'].median())








pond type count:
PondType
circular-big      93
circular-small    15
square-female     14
Name: count, dtype: int64
pond type mean:
PondType
circular-big      116.341756
circular-small     73.021133
square-female     119.587619
Name: Avg_Length, dtype: float64
pond type std:
PondType
circular-big      18.716363
circular-small    23.583171
square-female     17.805035
Name: Avg_Length, dtype: float64
min_mpe fov median:
PondType
circular-big       8.873005
circular-small    24.395622
square-female      1.922570
Name: min_mpe, dtype: float64
min_mae fov median:
PondType
circular-big      11.390734
circular-small    20.878955
square-female      2.386082
Name: min_mae, dtype: float64
max_mpe fov median:
PondType
circular-big      23.144713
circular-small    47.090420
square-female     12.111046
Name: max_mpe, dtype: float64
max_mae fov median:
PondType
circular-big      24.212128
circular-small    30.365965
square-female     15.193373
Name: max_mae, dtype: float64
min_mpe focal length median

In [14]:
#mean difference between min max of Length_1, Length_2, Length_3 for each pond type
print('mean difference between min max for each pond type:')
print(df.groupby('PondType')['diff_max_min'].mean())

mean difference between min max for each pond type:
PondType
circular-big      14.402247
circular-small     9.966533
square-female     14.834071
Name: diff_max_min, dtype: float64


In [16]:
#print uncertainty of the mean 
print('uncertainty of the mean:')
print(df.groupby('Pond_Type')['Std_Length'].mean()/np.sqrt(df.groupby('Pond_Type')['Avg_Length'].count()))

uncertainty of the mean:
Pond_Type
circular big male      0.261743
circular small male    0.649817
square-femal           0.428996
dtype: float64


In [ ]:
X = df[['camera_distance', 'pose_metric_1', 'pose_metric_2']]  # Add other pose metrics if available
X = sm.add_constant(X)
y = df['diff']
model = sm.OLS(y, X).fit()
print(model.summary())




melted_df = df.melt(id_vars=['Label','PrawnID'], value_vars=['Length_1', 'Length_2', 'Length_3'],
                   var_name='rater', value_name='measurement')
icc = pg.intraclass_corr(data=melted_df, targets=['Label','PrawnID'], raters='rater', ratings='measurement')
print(icc)


# Correlation Analysis
pearson_corr, pearson_p = pearsonr(df['keypoint_mm'], df['expert_mean'])
spearman_corr, spearman_p = spearmanr(df['keypoint_mm'], df['expert_mean'])
print(f"Pearson Correlation: {pearson_corr:.3f}, p-value: {pearson_p:.3e}")
print(f"Spearman Correlation: {spearman_corr:.3f}, p-value: {spearman_p:.3e}")

mean_measurements = (df['keypoint_mm'] + df['expert_mean']) / 2
diff_measurements = df['diff']
md = np.mean(diff_measurements)
sd = np.std(diff_measurements)

plt.figure(figsize=(10, 6))
plt.scatter(mean_measurements, diff_measurements, alpha=0.5)
plt.axhline(md, color='gray', linestyle='--', label=f'Mean Diff = {md:.2f} mm')
plt.axhline(md + 1.96*sd, color='red', linestyle='--', label=f'+1.96 SD = {md + 1.96*sd:.2f} mm')
plt.axhline(md - 1.96*sd, color='red', linestyle='--', label=f'-1.96 SD = {md - 1.96*sd:.2f} mm')
plt.xlabel('Mean of Measurements (mm)')
plt.ylabel('Difference (Keypoint - Expert) (mm)')
plt.title('Bland-Altman Plot')
plt.legend()
plt.show()



stat, p = shapiro(df['diff'])
print(f'Shapiro-Wilk Test: stat={stat:.3f}, p-value={p:.3e}')

# Homogeneity of Variance
# Assuming 'camera_distance_category' is already defined as in the visualization section
groups = df.groupby('camera_distance_category')['diff'].apply(list)
stat, p = levene(*groups)
print(f"Levene’s Test: stat={stat:.3f}, p-value={p:.3e}")
